In [14]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
config['augmentations']['pad_trunc_noise_multiplier']


'0.025'

In [1]:
import transforms

transforms.getTransforms(True)

c:\anaconda\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


[{},
 {'before_cochannel': [<audiomentations.augmentations.gain.Gain at 0x1a3803079d0>,
  'audio': [<audiomentations.augmentations.add_gaussian_noise.AddGaussianNoise at 0x1a3fdf56520>,
   <audiomentations.augmentations.pitch_shift.PitchShift at 0x1a380db5fd0>]},
 {'before_cochannel': [<audiomentations.augmentations.gain.Gain at 0x1a38055d850>,
  'spectrogram': [TimeMasking(), FrequencyMasking()]}]

In [45]:
import torch, torchaudio
import IPython.display as ipd

trimmed_audio, b = torchaudio.load(
    'E:/asdas.wav')

trimmed_audio = trimmed_audio[0][0:1000000]
print(trimmed_audio.shape)
trimmed_audio = torchaudio.functional.vad(trimmed_audio, b)
print(trimmed_audio.shape)
ipd.Audio(trimmed_audio, rate=b)

torch.Size([309446])
torch.Size([170246])


In [1]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from tqdm import tqdm

import torch
import torchaudio
import random
import utils
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import os

class AudioDataset(Dataset):
    def __init__(self, audio_paths, device):
        self.audio_paths = audio_paths
        self.device = device

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        waveform, sample_rate = torchaudio.load(self.audio_paths[idx])
        waveform = waveform[0].to(self.device)
        pad_len = 240000 - len(waveform)
        zero_pad = torch.zeros(pad_len, device=self.device)
        waveform = torch.concat((waveform, zero_pad),0)
        return waveform

audioPaths = list(Path('E:/Singapore Speech Corpus/Part 2.1').glob('**/*.wav'))
dataset = AudioDataset(audioPaths, torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"))
training_loader = torch.utils.data.DataLoader(dataset, batch_size=1024, shuffle=False, num_workers=0)



class MyPipeline(torch.nn.Module):
    def __init__(
        self,
        input_freq=16000,
        resample_freq=8000
    ):
        super().__init__()
        self.resample = torchaudio.transforms.Resample(orig_freq=input_freq, new_freq=resample_freq)

    def forward(self, waveform: torch.Tensor) -> torch.Tensor:
        # Resample the input
        processed = self.resample(waveform)
        processed = torch.split(processed, 4*8000, dim=1)
        return processed


pipeline = MyPipeline()

pipeline.to(device=torch.device("cuda"))

for batch in tqdm(training_loader):
    pipeline(batch)


  1%|          | 8/808 [00:08<14:35,  1.09s/it]


RuntimeError: Trying to create tensor with negative dimension -16640: [-16640]

In [87]:
a = torch.arange(16000).reshape(16,1000)
torch.split(a, 100, dim=1)

(tensor([[    0,     1,     2,  ...,    97,    98,    99],
         [ 1000,  1001,  1002,  ...,  1097,  1098,  1099],
         [ 2000,  2001,  2002,  ...,  2097,  2098,  2099],
         ...,
         [13000, 13001, 13002,  ..., 13097, 13098, 13099],
         [14000, 14001, 14002,  ..., 14097, 14098, 14099],
         [15000, 15001, 15002,  ..., 15097, 15098, 15099]]),
 tensor([[  100,   101,   102,  ...,   197,   198,   199],
         [ 1100,  1101,  1102,  ...,  1197,  1198,  1199],
         [ 2100,  2101,  2102,  ...,  2197,  2198,  2199],
         ...,
         [13100, 13101, 13102,  ..., 13197, 13198, 13199],
         [14100, 14101, 14102,  ..., 14197, 14198, 14199],
         [15100, 15101, 15102,  ..., 15197, 15198, 15199]]),
 tensor([[  200,   201,   202,  ...,   297,   298,   299],
         [ 1200,  1201,  1202,  ...,  1297,  1298,  1299],
         [ 2200,  2201,  2202,  ...,  2297,  2298,  2299],
         ...,
         [13200, 13201, 13202,  ..., 13297, 13298, 13299],
         [

In [ ]:
import pyaudio
import wave
# from torchaudio.io import StreamReader

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 8000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    print(data)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


In [3]:
import Augmentation
import torch
from AudioDataset import transformData
from torch.utils.data import Dataset
import audiomentations
from transforms import getTransforms

audio_paths = Augmentation.getAudioPaths("E:/Processed Audio/SPEECH/SPEECH 2.1")

audio_train_dataset = transformData(audio_paths, transformParams=getTransforms(True), generateCochannel=True)

test_dataloader = torch.utils.data.DataLoader(
    audio_train_dataset,
    batch_size=64,
    num_workers=0,
    shuffle=True,
)

batch = next(iter(test_dataloader))
batch

[tensor([[[[  2.7025,   2.6179,   2.6501,  ...,   2.5112,   2.7209,   2.7083],
           [  0.7660,   0.6380,   0.6489,  ...,   0.5043,   0.8471,   0.7526],
           [-12.1424,  -6.5759,  -6.4142,  ..., -12.9068,  -6.2648, -10.3065],
           ...,
           [ -3.5618,  -7.8071,  -7.5418,  ...,  -7.5102,  -6.6485, -11.3327],
           [ -4.8676,  -7.6796,  -8.6246,  ...,  -6.7987,  -7.8514, -11.8427],
           [ -7.3386, -13.6290, -12.1293,  ...,  -8.1418,  -7.8613, -10.4566]]],
 
 
         [[[  0.9945,   0.6891,   0.5769,  ...,   0.5830,   0.7183,   0.7996],
           [ -0.8219,  -1.2429,  -1.2649,  ...,  -1.4056,  -1.3557,  -1.3950],
           [ -9.2123,  -7.6392,  -6.5884,  ..., -11.7159,  -6.8823, -10.1842],
           ...,
           [-19.3397, -10.8613, -11.2212,  ..., -10.0969,  -7.5830, -12.8559],
           [ -9.2245, -10.0827,  -7.3946,  ..., -10.7909,  -7.4959, -13.7045],
           [-10.2137, -10.4279,  -6.5198,  ...,  -9.7559,  -7.4312, -11.1104]]],
 
 
        

In [23]:
from pydub import AudioSegment
import time
from pathlib import Path

def get_audio_duration_in_seconds(sound_file):
    return len(sound_file) / (1000 * 60)


start_time = time.time()

input_file_path = 'E:/Processed Audio/ENV/Recorded/0_0.wav'
output_file_path = './output.wav'
Path(input_file_path).parents[1] == Path('E:/Processed Audio/ENV')
# sound_file = AudioSegment.from_wav(input_file_path)

True

In [17]:
sound_dBFS = sound_file.dBFS
stripped_audio = AudioSegment.strip_silence(
    sound_file, silence_len=1100, silence_thresh=sound_dBFS - 10, padding=200)
